### load imports

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseDataWarehouse.db
conn = sqlite3.connect('CourseDataWarehouse.db')

In [3]:
%%sql
DROP TABLE IF EXISTS Courses;
DROP TABLE IF EXISTS Course_Meeting;
DROP TABLE IF EXISTS Catalog;
DROP TABLE IF EXISTS Program;
DROP TABLE IF EXISTS Fees;


DROP TABLE IF EXISTS IMPORT_COURSE_OFFERINGS;
DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS;
DROP TABLE IF EXISTS IMPORT_CATALOG_COURSES;


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

##### we loaded the CSV files in the terminal window

##### Create ERD tables

In [4]:
%%sql

CREATE TABLE Courses (
    CRN SMALLINT,
    Term VARCHAR(20),
    Section VARCHAR(10),
    Cat_ID VARCHAR(10),
    Faculty_Name VARCHAR(10),
    Credits TINYINT,
    Capacity SMALLINT,
    Actual SMALLINT,
    Remaining SMALLINT,
    PRIMARY KEY (CRN, Term),
    FOREIGN KEY(Cat_ID) REFERENCES Catalog(Cat_ID)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [5]:
%%sql

CREATE TABLE Course_Meeting (
    MID INTEGER PRIMARY KEY,
    CRN SMALLINT,
    Term VARCHAR(20),
    Room TEXT,
    Day TEXT,
    Start_Time TIME,
    End_Time TIME,  
    FOREIGN KEY(CRN,Term) REFERENCES Courses(CRN,Term)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [6]:

%%sql

CREATE TABLE Catalog (
    Cat_ID VARCHAR(10) PRIMARY KEY,
    Prg_Code VARCHAR(5),
    Title VARCHAR(150),
    Prerequisites VARCHAR(10),
    Corequisites VARCHAR(10),
    FOREIGN KEY(Prg_Code) REFERENCES Program(Prg_Code)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [7]:
%%sql

CREATE TABLE Program (
    Prg_Code VARCHAR(10) PRIMARY KEY,
    Program_Name VARCHAR(150)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [8]:
%%sql

CREATE TABLE Fees (
    Cat_ID VARCHAR(10),
    Academic_Year VARCHAR(20),
    Fee TEXT,
    PRIMARY KEY (Cat_ID, Academic_Year)
    FOREIGN KEY(Cat_ID) REFERENCES Catalog(Cat_ID)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [9]:

#Catalog Data
catalog_years = ['2017_2018', '2018_2019']

for cat_year in catalog_years:
    filepath = '../SourceData/Catalogs/CourseCatalog'+cat_year+'.csv'
    data = pd.read_csv(filepath)
    data['cat_year'] = cat_year
    data.to_sql('IMPORT_CATALOG_COURSES',conn,if_exists='append',index=False)

In [10]:
#Course Offering and Course Meeting Data
terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019',
        'SpringBreak2017',
         'Summer2015','Summer2016','Summer2017','Summer2018',
         'Winter2015','Winter2016','Winter2017','Winter2018']

for term in terms:
    filepath = '../SourceData/'+term+'/courses.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_OFFERINGS',conn,if_exists='append',index=False) 
    
    filepath = '../SourceData/'+term+'/course_meetings.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_MEETINGS',conn,if_exists='append',index=False)



##### Insert into ERD tables


In [11]:

%%sql

INSERT INTO Program (Prg_Code, Program_Name)
SELECT DISTINCT program_code, program_name 
FROM IMPORT_CATALOG_COURSES;

 * sqlite:///CourseDataWarehouse.db
83 rows affected.


[]

In [12]:
%%sql

INSERT INTO Catalog (Cat_ID, Prg_Code, Title, Prerequisites, Corequisites)
SELECT DISTINCT catalog_id, program_code, course_title,prereqs,coreqs
FROM IMPORT_CATALOG_COURSES 
    JOIN Program ON (Prg_Code = program_code);

 * sqlite:///CourseDataWarehouse.db
2220 rows affected.


[]

In [13]:
%%sql

INSERT INTO Course_Meeting (CRN, Term, Room, Day, Start_Time, End_Time)
SELECT DISTINCT crn, term, location, day, start, end
FROM IMPORT_COURSE_MEETINGS;


 * sqlite:///CourseDataWarehouse.db
311141 rows affected.


[]

In [14]:
%%sql

INSERT INTO Courses (CRN, Term, Section, Cat_ID, Faculty_Name, Credits, Capacity, Actual, Remaining)
SELECT DISTINCT IMPORT_COURSE_OFFERINGS.crn, IMPORT_COURSE_OFFERINGS.term, section, catalog_id, primary_instructor, credits, cap, act, rem
FROM IMPORT_COURSE_OFFERINGS
    LEFT JOIN Course_Meeting ON Course_Meeting.CRN = IMPORT_COURSE_OFFERINGS.crn AND Course_Meeting.Term = IMPORT_COURSE_OFFERINGS.term
    LEFT JOIN Catalog ON Catalog.Cat_ID = IMPORT_COURSE_OFFERINGS.catalog_id;

 * sqlite:///CourseDataWarehouse.db
15937 rows affected.


[]

In [15]:
%%sql

INSERT INTO Fees (Cat_ID, Academic_Year, Fee )
SELECT DISTINCT catalog_id, cat_year, fees
FROM IMPORT_CATALOG_COURSES
    LEFT JOIN Catalog ON Catalog.Cat_ID = IMPORT_CATALOG_COURSES.catalog_id;   

 * sqlite:///CourseDataWarehouse.db
4440 rows affected.


[]

In [16]:
%%sql

DROP TABLE IF EXISTS Location;
DROP TABLE IF EXISTS Course_Facts;
DROP TABLE IF EXISTS Meeting;
DROP TABLE IF EXISTS Fees;
DROP TABLE IF EXISTS Program1;


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.


[]

In [17]:
%%sql

CREATE TABLE Location (
    LID INTEGER PRIMARY KEY,
    Room TEXT
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [18]:
%%sql

INSERT INTO Location (Room)
SELECT DISTINCT Room 
FROM Course_Meeting;

 * sqlite:///CourseDataWarehouse.db
207 rows affected.


[]

In [19]:
%%sql

CREATE TABLE Program1 (
    PID INTEGER PRIMARY KEY,
    Prg_Code VARCHAR(10),
    Program_Name VARCHAR(150)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [20]:
%%sql

INSERT INTO Program1 (Prg_Code, Program_Name)
SELECT DISTINCT program_code, program_name 
FROM IMPORT_CATALOG_COURSES;

 * sqlite:///CourseDataWarehouse.db
83 rows affected.


[]

In [21]:
%%sql

CREATE TABLE Meeting (
    MID INTEGER PRIMARY KEY,
    Cat_ID TEXT,
    Day TEXT,
    Start_Time TIME,
    End_Time TIME  
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [22]:
%%sql

INSERT INTO Meeting (Day, Start_Time, End_Time)
SELECT DISTINCT day, start, end
FROM IMPORT_COURSE_MEETINGS


 * sqlite:///CourseDataWarehouse.db
42551 rows affected.


[]

In [23]:
%%sql

CREATE TABLE Fees (
    Fee_ID INTEGER PRIMARY KEY,
    Cat_ID VARCHAR(10),
    Academic_Year VARCHAR(20),
    Amount VARCHAR(50) 
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [24]:
%%sql

INSERT INTO Fees (Cat_ID, Academic_Year, Amount )
SELECT DISTINCT catalog_id, cat_year, fees
FROM IMPORT_CATALOG_COURSES;


 * sqlite:///CourseDataWarehouse.db
4440 rows affected.


[]

In [25]:
%%sql

CREATE TABLE Course_Facts (
    CRN SMALLINT,
    Term VARCHAR(20),
    Faculty_Name VARCHAR(10),
    Title VARCHAR(100),
    Section VARCHAR(10),
    Cat_ID VARCHAR(10),
    LID VARCHAR(10),
    MID VARCHAR(10),
    PID VARCHAR(10),
    FEE_ID VARCHAR(10),
    Capacity SMALLINT,
    Actual SMALLINT,
    Remaining SMALLINT
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [26]:
%%sql
INSERT INTO Course_Facts (CRN, Term, Faculty_Name,Title, Cat_ID,PID, Section,Capacity,Actual,Remaining, LID, MID, Fee_Id)
SELECT Courses.CRN, Courses.Term, Faculty_Name,Catalog.Title, Courses.Cat_ID, Program1.PID,  Section, Capacity, Actual, Remaining, Location.LID, Meeting.MID, Fees.Fee_ID
FROM Courses
    LEFT JOIN Catalog USING (Cat_ID)
    LEFT JOIN Program1 USING (Prg_Code)
    LEFT JOIN Course_Meeting ON Course_Meeting.CRN = Courses.CRN AND Course_Meeting.Term = Courses.Term
    LEFT JOIN Location ON Course_Meeting.Room = Location.Room
    LEFT JOIN Meeting ON Course_Meeting.Start_Time = Meeting.Start_Time
    LEFT JOIN Fees on Fees.Cat_ID = Catalog.Cat_ID;
    
    

 * sqlite:///CourseDataWarehouse.db
2289550 rows affected.


[]

In [27]:
%%sql
DROP TABLE IF EXISTS Courses;
DROP TABLE IF EXISTS Course_Meeting;
DROP TABLE IF EXISTS Catalog;
DROP TABLE IF EXISTS Program;
DROP TABLE IF EXISTS Fees;


DROP TABLE IF EXISTS IMPORT_COURSE_OFFERINGS;
DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS;
DROP TABLE IF EXISTS IMPORT_CATALOG_COURSES;


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [28]:
%%sql
DROP TABLE IF EXISTS Course_Facts;
DROP TABLE IF EXISTS Meeting;
DROP TABLE IF EXISTS Fees;
DROP TABLE IF EXISTS Program1;
DROP TABLE IF EXISTS Location;





 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.


[]

In [29]:
%%sql
Vacuum;

 * sqlite:///CourseDataWarehouse.db
Done.


[]